# DPO

In [1]:
!pip install transformers accelerate hf-transfer peft - Uq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 44.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
# policy model (dpo 학습대상)
# ref model (참조모델, 학습 대상아님)
model_id = "LGAI-EXAONE/EXAONE-4.0-1.2B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/332 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
# 샘플데이터
prompt = "파이썬이 뭐야?"
chosen = "파이썬은 배우기 쉽고, 강력한 프로그래밍 언어입니다."
rejected = "파이썬은 뱀이지~"


In [25]:
# 생성확률 계산
# - 모델이 답변을 실제 생성하는게 아니라
# - 데이터셋에서 주어진 텍스트(토큰)에 대한 생성확률을 계산하고,
# - chosen데이터의 생성확률은 높히고, rejected데이터의 생성확률은 낮추는 학습


def get_logprob(model, tokenizer, prompt, response):
    """답변의 생성확률(로그) 계산하는 함수"""
    full_text = prompt + response
    inputs = tokenizer(full_text, return_tensors="pt").to(model.device)
    prompt_len = len(tokenizer(prompt, return_tensors="pt")["input_ids"][0])
    # print(full_text)
    # print(len(inputs['input_ids'][0]))
    # print(prompt_len)

    with torch.no_grad():
        outputs = model(**inputs)  # (batch_size, seq_len, vocab_size)
        logits = outputs.logits
        # print(logits.shape)
        log_probs = F.log_softmax(logits, dim=-1)  # log(확률변환)
        # print(log_probs.shape)

        # print(f'생성할 토큰의 확률모음: {log_probs[:, :-1].shape}')
        # print(f'정답 토큰 id: {inputs['input_ids'][:, 1:].unsqueeze(-1).shape}')
        # 생성토큰의 확률추출
        token_log_probs = (
            log_probs[:, :-1]
            .gather(
                index=inputs["input_ids"][:, 1:].unsqueeze(-1), dim=-1  # 정답토큰id
            )
            .squeeze(-1)
        )
        # print(f'정답 토큰의 생성확률: {token_log_probs.shape}')

        # response부분만 추출(prompt 제외)
        response_log_probs = token_log_probs[:, prompt_len - 1 :]
        # 누적 생성확률
        total = response_log_probs.sum()

    return total.item()


chosen_prob = get_logprob(model, tokenizer, prompt, chosen)
rejected_prob = get_logprob(model, tokenizer, prompt, rejected)
print(f"선택된 답변 생성확률: {chosen_prob}")
print(f"거절된 답변 생성확률: {rejected_prob}")


생성할 토큰의 확률모음: torch.Size([1, 19, 102400])
정답 토큰 id: torch.Size([1, 19, 1])
정답 토큰의 생성확률: torch.Size([1, 19])
생성할 토큰의 확률모음: torch.Size([1, 11, 102400])
정답 토큰 id: torch.Size([1, 11, 1])
정답 토큰의 생성확률: torch.Size([1, 11])
선택된 답변 생성확률: -85.0
거절된 답변 생성확률: -59.0


In [24]:
# dpo 손실함수
# - 정책모델, 참조모델의 차이를 작게 만들려는 목적(손실)함수를 사용
def dpo_loss(policy_chosen, policy_rejected, ref_chosen, ref_rejected, beta=0.1):
    logits = beta * ((policy_chosen - policy_rejected) - (ref_chosen - ref_rejected))
    loss = -F.logsigmoid(torch.tensor(logits))
    return loss.item()